In [2]:
import math
import copy
import json
import os
import shutil
import cv2
#pip install opencv-contrib-python
import time
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Image
from subprocess import run
import open3d as o3d
#pip install open3d
from pyntcloud import PyntCloud
#pip install pyntcloud

In [3]:
#Traductor de rotacion
class node:
    t = 0 
    ro = 0 
    tx = 0
    ty = 0
    tz = 0
    rx = 0
    ry = 0
    rz = 0

    def __init__(self, shape='set', operation='None',l=0, b=0, h=0,fn=12, fa=12, fs=2, r=0, r1=0,center="true"):
        self.left = 'None'
        self.right = 'None'
        self.string = ''
        self.operation = operation
        self.shape = shape
        self.l = l
        self.b = b
        self.h = h
        self.fn = fn
        self.fa = fa
        self.fs = fs
        self.r = r 
        self.r1 = r1
        self.center = center

    def translate(self, x=0, y=0, z=0):
        self.tx = x
        self.ty = y
        self.tz = z
        self.t = 1

    def rotate(self, x=0, y=0, z=0):
        self.rx = x
        self.ry = y
        self.rz = z
        self.ro = 1

    def name(self):
        if(self.t == 1):
            self.string = 'translate([' + str(self.tx) + ',' + str(self.ty) + ',' + str(self.tz) + ']){\n\t'
        if(self.ro == 1):
            if(self.t == 1):
                self.string = self.string + 'rotate([' + str(self.rx) + ',' + str(self.ry) + ',' + str(self.rz) + ']){\n\t\t'
            else:
                self.string = self.string +'rotate([' + str(self.rx) + ',' + str(self.ry) +',' + str(self.rz) + ']){\n\t'
        if(self.shape == "cube"):
            self.string = self.string + 'cube([' + str(self.l) + ',' + str(self.b) + ',' + str(self.h) + '], center = ' + self.center + ');\n'
        if(self.shape == "cylinder"):
            self.string = self.string + 'cylinder($fn = ' +str(self.fn)+', h = '+str(self.h)+', r1 = '+str(self.r)+', r2 = ' + str(self.r1) + ', center = ' + self.center + ');\n'
        if(self.shape == "sphere"):
            self.string = self.string + 'sphere($fn = ' + str(self.fn) + ', $fa = ' + str(self.fa) + ', $fs = ' + str(self.fs) + \
                'r = ' + str(self.r) + ');\n'
        if(self.ro == 1 and self.t == 1):
            self.string = self.string + "\t}\n"
        if(self.ro == 1 or self.t == 1):
            self.string = self.string + "}\n"

    def setr(self, r):
        self.r = r

    def setl(self, l):
        self.l = l

    def setb(self, b):
        self.b = b

    def seth(self, h):
        self.h = h

    def setr1(self, r):
        self.r1 = r

    def setoperation(self, operation):
        self.operation = operation

In [4]:
#Arbol binario
def writetofile(filename, string):
    file = open(filename, "a")
    file.write(string)
    file.close()

def createtree(nodes, filename):
    if(len(nodes) == 0):
        print("No shapes detected")
        return
    subtree = []
    string = ''
    present = node()
    present.left = nodes[0]
    nodes[0].name()
    present.string = nodes[0].string
    if(len(nodes) == 1):
        writetofile(filename, present.string)
    for i, object in enumerate(nodes[1:]):
        object.name()
        if(object.operation != 'None'):
            if(present.right == 'None'):
                present.right = object
                present.string = object.operation + "() {\n\t" + present.string + '\n\t' + object.string + '}\n'
                string = present.string
            else:
                temp = present
                present = node()
                present.left = temp
                present.string = temp.string
                present.right = object
                present.string = object.operation + "() {\n\t" + present.left.string + '\n\t' + object.string + '}\n'
                string = present.string
        else:
            if(present.right == 'None'):
                writetofile(filename, present.string)
            subtree.append(present)
            writetofile(filename, string)
            string = ''
            present = node()
            present.left = object
            present.string = object.string
            if(i == len(nodes)-2):
                writetofile(filename, present.string)
    if(string != ''):
        writetofile(filename, string)
    subtree.append(present)
    print("Subtree:", len(subtree))

In [5]:
#Multiples Operaciones
def round_up(n, decimals=2):
    multiplier = 10 ** decimals
    n = math.ceil(n * multiplier) / multiplier
    return n

def re_arrange(objects, type):
    if(type == "front"):
        objects.sort(key=lambda x: x[0][1][0], reverse=True)
    else:
        objects.sort(key=lambda x: x[0][1][0])

def draw_contour(img, out):
    imgrey = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgrey, 127, 255, cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy, contours = zip(*sorted(zip(hierarchy[0], contours), key=lambda x: cv2.contourArea(x[1]), reverse=True))
    for i, c in enumerate(contours):
        if (hierarchy[i][3] != -1 or (hierarchy[i][3] == -1 and hierarchy[i][2] == -1)):
            M = cv2.moments(c)
            if(M["m00"] != 0):
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                area = cv2.contourArea(c)
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                cv2.drawContours(img, [c], 0, (0, 0, 255), 1)
    cv2.imwrite(out, img)

def add_part(objects, part, pos, type, ratio):
    if(type == "front"):
        part[0].translate(round_up((part[1][0]-pos[0]) * ratio),0, round_up((pos[1]-part[1][1]) * ratio))
    if(type == "side"):
        part[0].translate(0, round_up((part[1][0]-pos[0]) * ratio),round_up((pos[1]-part[1][1]) * ratio))
    if(type == "top"):
        part[0].translate(round_up((part[1][1]-pos[1]) * ratio),round_up((part[1][0]-pos[0]) * ratio), 0)
    for i, object in enumerate(objects):
        if(cv2.pointPolygonTest(object[0][2], part[1], False) >= 0):
            part[0].operation = "difference"
            object.append(part)
            return
    objects.append([part])
    return

def combining(front_parts, side_parts, top_parts, roundOffApprox):
    for i, front in enumerate(front_parts):
        first = 0
        for j, part_f in enumerate(front):
            if(len(part_f) >= 6):
                continue
            if(first == 1):
                if(part_f[0].shape == "cube"):
                    if(front[0][0].shape == "cube"):
                        part_f[0].b = front[0][0].b
                    else:
                        part_f[0].b = front[0][0].h
                    if(front[0][0].rz != 0):
                        part_f[0].b = front[0][3]
                if(part_f[0].shape == "cylinder"):
                    if(front[0][0].shape == "cube"):
                        part_f[0].h = front[0][0].b
                    else:
                        part_f[0].h = front[0][0].h
                    if(front[0][0].rz != 0):
                        part_f[0].h = front[0][3]
            found = 0
            for k, side in enumerate(side_parts):
                add_rest = 0
                for l, part_s in enumerate(side):
                    if(add_rest == 1 and part_s[3] != -1):
                        part_s.append("used")
                        part_s[3] = -1
                        part_s[0].tx = part_f[0].tx
                        if(part_s[0].shape == "cylinder"):
                            part_s[0].h = part_f[4]
                        elif(part_s[0].shape == "cube"):
                            part_s[0].b = part_f[4]
                        front.append(part_s)
                        continue
                    if(abs(part_f[3]-part_s[3]) < roundOffApprox and found == 0):
                        found = 1
                        if(j == 0):
                            first = 1
                        if(l != 0):
                            part_s[0].operation = "union"
                        if(part_f[0].shape == "cube"):
                            if(part_s[0].shape == "cube"):
                                for top in top_parts:
                                    for m, part_t in enumerate(top):
                                        if(abs(part_t[3] - part_f[4]) < roundOffApprox and len(part_t) == 5):
                                            if(m != 0):
                                                part_t[0].operation = "union"
                                            if(j == 0):
                                                part_t[0].operation = "None"
                                            if(part_t[0].shape == "cylinder"):
                                                part_t[0].h = part_f[0].h
                                                part_t[0].tz = part_f[0].tz
                                                part_t[1] = i
                                                part_t[2] = -1
                                                part_t.append("used")
                                                part_f[0] = part_t[0]
                                                part_s[3] = -1
                                                add_rest = 1
                                                break
                                            if(part_t[0].shape == "cube" and part_s[0].ry == 0 and part_f[0].ry == 0 and part_t[0].rz != 0 and abs(part_t[0].rz) != 90):
                                                part_t[0].b = part_f[0].h
                                                part_t[0].tz = part_f[0].tz
                                                part_t[1] = i
                                                part_t[2] = -1
                                                part_t.append("used")
                                                part_f[0] = part_t[0]
                                                part_f[3] = part_t[3]
                                                part_s[3] = -1
                                                add_rest = 1
                                                break
                                    else:
                                        continue
                                    break
                            if(part_s[3] == -1):
                                continue
                            if(part_s[0].shape == "cylinder"):
                                part_s[0].h = part_f[0].l
                                part_s[0].tx = part_f[0].tx
                                part_s[0].ry = part_f[0].ry
                                if(j == 0):
                                    part_s[0].operation = "None"
                                part_f[0] = part_s[0]
                                part_s[3] = -1
                                add_rest = 1
                                continue
                            if(part_s[0].shape == "cube"):
                                if(part_f[0].ry != 0 or part_s[0].ry == 0):
                                    part_f[0].b = part_s[0].l
                                    part_f[0].ty = part_s[0].ty
                                    part_s[3] = -1
                                    add_rest = 1
                                    continue
                                else:
                                    part_s[0].b = part_f[0].l
                                    part_s[0].tx = part_f[0].tx
                                    part_f[0] = part_s[0]
                                    part_s[3] = -1
                                    add_rest = 1
                                    continue
                        if(part_f[0].shape == "cylinder"):
                            if(part_s[0].shape == "cube"):
                                part_f[0].h = part_s[0].l
                                part_f[0].ty = part_s[0].ty
                                part_f[0].rx = part_s[0].ry
                        part_s[3] = -1
                        add_rest = 1
                        continue
                if(found == 1):
                    break
            if(found == 0):
                for top in top_parts:
                    for m, part_t in enumerate(top):
                        if((abs(part_t[3]-part_f[4]) < roundOffApprox) and len(part_t) == 5):
                            if(abs(part_f[0].tx - part_t[0].tx) > 0.5):
                                continue
                            if(j == 0):
                                first = 1
                            if(m != 0):
                                part_t[0].operation = "union"
                            if(part_f[0].shape == "cylinder"):
                                if(part_t[0].shape == "cube"):
                                    part_f[0].h = part_t[0].h
                                    part_f[0].ty = part_t[0].ty
                                    part_f[0].rz = part_t[0].rz
                                    part_t[0].b = part_f[3]
                                    part_t[0].tz = part_f[0].tz
                                    part_t[1] = i
                                    part_t[2] = -1
                                    part_t.append("used")
                                    break
                            if(part_f[0].shape == "cube"):
                                part_t[0].tz = part_f[0].tz
                                if(part_t[0].shape == "cube"):
                                    part_t[0].b = part_f[0].h
                                    if(part_t[0].rz != 0):
                                        part_f[0] = part_t[0]
                                    else:
                                        part_f[0].b = part_t[0].l
                                        part_f[0].ty = part_t[0].ty
                                elif(part_t[0].shape == " cylinder"):
                                    part_t[0].ry = part_f[0].ry
                                    part_t[0].h = part_f[0].h
                                    part_f[0] = part_t[0]
                                part_t[1] = j
                                part_t[2] = -1
                                part_t.append("used")
                                break
                    else:
                        continue
                    break
    for side in side_parts:
        if(side[0][3] != -1):
            addCompleteList = []
            for part_s in side:
                if(part_s[3] == -1):
                    break
                for k, top in enumerate(top_parts):
                    for l, part_t in enumerate(top):
                        if((abs(part_s[4]-part_t[4]) < roundOffApprox) and len(part_t) == 5):
                            if(part_s[0].shape == "cube"):
                                part_t[0].tz = part_s[0].tz
                                if(part_t[0].shape == "cube"):
                                    part_t[0].b = part_s[0].h
                                    if(part_t[0].rz != 0):
                                        part_s[0] = part_t[0]
                                    else:
                                        part_s[0].b = part_t[0].h
                                        part_s[0].tx = part_t[0].tx
                                elif(part_t[0].shape == " cylinder"):
                                    part_t[0].rx = part_s[0].ry
                                    part_t[0].h = part_s[0].h
                                    part_s[0] = part_t[0]
                                part_t[1] = len(front_parts)
                                part_t[2] = -1
                                part_t.append("used")
                            if(part_s[0].shape == "cylinder"):
                                if(part_t[0].shape == "cube"):
                                    part_s[0].h = part_t[0].h
                                    part_s[0].tx = part_t[0].tx
                                    part_s[0].rz = part_t[0].rz
                                    part_t[0].b = part_s[3]
                                    part_t[0].tz = part_s[0].tz
                                    part_t[1] = len(front_parts)
                                    part_t[2] = -1
                                    part_t.append("used")
                addCompleteList.append(part_s)
            front_parts.append(addCompleteList)
    for top in top_parts:
        height = -1
        tz = -1
        position = -1
        for part_t in top:
            if(height != -1 and len(part_t) == 5):
                if(part_t[0].shape == "cylinder"):
                    part_t[0].h = height
                    part_t[0].tz = tz
                elif (part_t[0].shape == "cube"):
                    part_t[0].b = height
                    part_t[0].tz = tz
                try:
                    front_parts[position].append(part_t)
                except:
                    print("Error occurred")
            try:
                class Spam(int):
                    pass
                if(isinstance(Spam(part_t[2]), int)):
                    tz = part_t[0].tz
                    position = part_t[1]
                    if(part_t[0].shape == "cylinder"):
                        height = part_t[0].h
                    elif (part_t[0].shape == "cube"):
                        height = part_t[0].b
            except TypeError:
                pass
    return front_parts

def detect(c):
    shape = "unidentified"
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    cylinder_type = 0
    if len(approx) == 3:
        shape = "triangle"
        cylinder_type = 3
    elif len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        ar = w / float(h)
        rect = cv2.minAreaRect(c)
        ar = rect[1][0] / float(rect[1][1])
        shape = "square" if ar >= 0.999 and ar <= 1.001 else "rectangle"
    elif len(approx) == 5:
        shape = "pentagon"
        cylinder_type = 5
    elif len(approx) == 6:
        shape = "hexagon"
        cylinder_type = 6
    else:
        shape = "circle"
        cylinder_type = 1
    return shape, cylinder_type

def valid_contours(img, type, ratio):
    objects = []
    imgrey = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgrey, 127, 255, cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy, contours = zip(*sorted(zip(hierarchy[0], contours), key=lambda x: cv2.contourArea(x[1]), reverse=True))
    for i, c in enumerate(contours):
        if (hierarchy[i][3] != -1 or (hierarchy[i][3] == -1 and hierarchy[i][2] == -1)):
            M = cv2.moments(c)
            if(M["m00"] != 0):
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                area = cv2.contourArea(c)
                area = area * ratio
                rect = cv2.minAreaRect(c)
                if(area <= 4*ratio):
                    break
                box = cv2.boxPoints(rect)
                rectLength = round_up(rect[1][0] * ratio)
                rectBreadth = round_up(rect[1][1] * ratio)
                x, y, w, h = cv2.boundingRect(c)
                w = round_up(w * ratio)
                h = round_up(h * ratio)
                if(len(objects) == 0):
                    pos = [cX, cY]
                cv2.drawContours(img, [c], 0, (0, 0, 255), 1)
                shape, cylinder_type = detect(c)
                part = []
                if(shape == "square"):
                    part.append(node(shape="cube", l=rectLength,b=rectLength, h=rectLength))
                    if(type == "front"):
                        part[0].rotate(0, rect[2], 0)
                    elif(type == "side"):
                        part[0].rotate(0, rect[2], 90)
                    elif(rect[2] == 0):
                        part[0].rotate(90, 0, 90)
                    else:
                        part[0].rotate(90, 0, rect[2])
                    part.append(tuple([cX, cY]))
                    part.append(c)
                    part.append(h)
                    part.append(w)
                    add_part(objects, part, pos, type, ratio)
                elif(shape == "rectangle"):
                    if(type == "front"):
                        if(rect[2] == -90):
                            part.append(node(shape="cube", l=rectBreadth, b=rectBreadth, h=rectLength))
                            part[0].rotate(0, 0, 0)
                        else:
                            part.append(node(shape="cube", l=rectLength, b=rectLength, h=rectBreadth))
                            part[0].rotate(0, rect[2], 0)
                    elif(type == "side"):
                        if(rect[2] == -90):
                            part.append(node(shape="cube", l=rectBreadth, b=rectBreadth, h=rectLength))
                            part[0].rotate(0, 0, 90)
                        else:
                            part.append(node(shape="cube", l=rectLength, b=rectLength, h=rectBreadth))
                            part[0].rotate(0, rect[2], 90)
                    elif(rect[2] == -90):
                        part.append(node(shape="cube", l=rectBreadth,b=rectBreadth, h=rectLength))
                        part[0].rotate(90, 0, 90)
                    elif(rect[2] == 0):
                        part.append(node(shape="cube", l=rectLength,b=rectLength, h=rectBreadth))
                        part[0].rotate(90, 0, 90)
                    else:
                        part.append(node(shape="cube", l=rectLength,b=rectLength, h=rectBreadth))
                        part[0].rotate(90, 0, rect[2])
                    part.append(tuple([cX, cY]))
                    part.append(c)
                    part.append(h)
                    part.append(w)
                    add_part(objects, part, pos, type, ratio)
                elif(cylinder_type > 0):
                    if(shape == "circle"):
                        _, radius = cv2.minEnclosingCircle(c)
                        radius = round_up(radius * ratio)
                        part.append(node(shape="cylinder", r=radius, r1=radius, h=rectLength))
                        if(type == "front"):
                            part[0].rotate(90, 0, 0)
                        elif(type == "side"):
                            part[0].rotate(0, 90, 0)
                        part.append(tuple([cX, cY]))
                        part.append(c)
                        part.append(h)
                        part.append(w)
                        add_part(objects, part, pos, type, ratio)
                    else:
                        _, radius = cv2.minEnclosingCircle(c)
                        radius = round_up(radius * ratio)
                        part.append(node(shape="cylinder", r=radius,r1=radius, h=rectLength, fn=cylinder_type))
                        if(type == "front"):
                            part[0].rotate(90, -90 - rect[2], 0)
                        elif(type == "side"):
                            part[0].rotate(0, 90, 0)
                        else:
                            part[0].rotate(0, 0, rect[2])
                        part.append(tuple([cX, cY]))
                        part.append(c)
                        part.append(h)
                        part.append(w)
                        add_part(objects, part, pos, type, ratio)
                else:
                    print("shape not detected")
    return objects

In [6]:
print("Please enter the image view file with extension")
s=input()
image=cv2.imread(s)
imgray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray,127,255,0,cv2.THRESH_BINARY_INV)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
hierarchy,contours = zip(*sorted(zip(hierarchy[0],contours),key = lambda x: cv2.contourArea(x[1]),reverse=True))
print ("no, of contours = " + str(len(contours)-1))
cv2.drawContours(image, contours, 1, (0, 255, 0), 2)
cv2.drawContours(image, contours, 2, (0, 0 , 255), 2)
cv2.imshow('IMAGE', image)
cv2.imshow('Image Gray', imgray)
cv2.waitKey(0)

Please enter the image view file with extension


[ WARN:0@1484.026] global /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('front.png'): can't open/read file: check file path/integrity


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
#Front
print("Please enter the image view file with extension")
s=input()
image=cv2.imread(s)
imgray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray,127,255,0,cv2.THRESH_BINARY_INV)
contours, hierarchy = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_NONE)
hierarchy,contours = zip(*sorted(zip(hierarchy[0],contours),key = lambda x: cv2.contourArea(x[1]),reverse=True))

front.png
Below Function 'detect()' will determine the shape of a given 2D image on the basis of number of edges in the given Shape .

Allowed Shapes for a 2d Image are : Triangle , Square , Rectangle , Pentagon , Hexagon , Circle

Extension .png

In [ ]:
def detect(c):
    perimeter = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * perimeter , True)
    if len(approx) == 3:
        shape = "triangle"
    elif len(approx) == 4:
        (x, y, w, h) = cv2.boundingRect(approx)
        ar = w / float(h)
        if (ar >= 0.999 and ar <= 1.001):
            shape = "square" 
        else:
            shape = "rectangle"
    elif len(approx) == 5:
        shape = "pentagon"
    elif len(approx) == 6:
        shape = "hexagon"
    else:
        shape = "circle"
    return shape

In [ ]:
d=detect(contours[1])
print(d)

In [ ]:
def Dimensioning(userId,view,image):
    os.makedirs('static/temp', exist_ok=True)
    img = cv2.imread(image,cv2.IMREAD_UNCHANGED)
    ratio = 0
    shape = "unidentified"
    try:
        w,h,_=img.shape
        drawSize = int(h/300)
        imgrey = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgrey,127,255,cv2.THRESH_BINARY_INV)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
        hierarchy,contours = zip(*sorted(zip(hierarchy[0],contours),key = lambda x: cv2.contourArea(x[1]),reverse=True))
        for i,c in enumerate(contours):
            if(hierarchy[i][3] != -1  or (hierarchy[i][3] == -1 and hierarchy[i][2] == -1) ):
                M = cv2.moments(c)
                if(M["m00"] !=0):
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    rect = cv2.minAreaRect(c)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    x,y,w,h = cv2.boundingRect(c)
                    shape, cylinder_type = detect(c)              
                    if(shape == "unidentified"):
                        continue
                    if(shape=="triangle" or shape=="pentagon" or shape=="hexagon"):
                        img = cv2.drawContours(img, [box], 0, (0,0, 255), drawSize)
                    if(shape=="circle"):
                        img = cv2.rectangle(img,(x,y),(x+w,y+h),(0, 0, 255),drawSize)
                        cv2.line(img, (x,y), (x+w,y), (0,255, 0), 2)
                        ratio = 1/w
                    else:
                        cv2.line(img, tuple(box[0]), tuple(box[1]), (0,255, 0), 2)
                        ratio = 1.0/rect[1][1]
                    break
        folder = 'static/temp/' + userId
        os.makedirs(folder, exist_ok=True)
        try:
            os.remove(folder + '/' + view + '.jpg')
        except: pass
        path_file = (folder + '/' + view + '.jpg')
        small = cv2.resize(img, (0,0), fx=0.5, fy=0.5) 
        cv2.imwrite(path_file,small)
        ratio = str(ratio)
        data={'image': path_file,'shape': shape,'ratio': ratio}
        display(Image(filename = path_file))
        return data
    except:
        print(view + "image not found")

In [ ]:
def Convert(userId, front_image, side_image, top_image, fratio, sratio, tratio): 
    start = time.time() 
    os.makedirs('static/'+userId, exist_ok=True)
    img_front = cv2.imread(front_image,cv2.IMREAD_UNCHANGED)
    img_side = cv2.imread(side_image,cv2.IMREAD_UNCHANGED)
    img_top = cv2.imread(top_image,cv2.IMREAD_UNCHANGED)
    fratio = float(fratio)
    sratio = float(sratio)
    tratio = float(tratio)
    filePath = "static/temp/"+userId
    try:
        shutil.rmtree(filePath)
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))
    primitive = []
    object_front = valid_contours(img_front,"front",fratio)
    re_arrange(object_front,"front")
    object_side = valid_contours(img_side,"side",sratio)
    re_arrange(object_side,"side")
    object_top = valid_contours(img_top,"top",tratio)
    re_arrange(object_top,"top")
    minApprox = 0.05
    primitive = combining(object_front,object_side,object_top,minApprox)
    final = []
    for set in primitive:
        for shape in set:
            final.append(shape[0])
    try:
        os.remove('static/' + userId + "/" + userId + '.scad')
    except: pass
    path_file = ('static/' + userId + "/" + userId + '.scad')
    if(len(final) == 0):
        path_file = 'static/error.txt'
        f = open(path_file, "w")
        f.write("Cannot determine the 3d geometry, check your files again!")
        f.close()
    createtree(final,path_file)
    end = time.time() 
    print("Total time taken to convert:",end-start)
    return path_file

In [ ]:
userId="1"
front_image = "TestBench/"+userId+"/front.jpg"
side_image = "TestBench/"+userId+"/side.jpg"
top_image = "TestBench/"+userId+"/top.jpg"
front_image

In [ ]:
d1 = Dimensioning(userId,"front",front_image)
d1

In [ ]:
d2 = Dimensioning(userId,"side",side_image)
d2

In [ ]:
d3 = Dimensioning(userId,"top",top_image)
d3

In [ ]:
fratio = float(d1["ratio"]) * 2 
sratio = float(d2["ratio"]) * 2
tratio = float(d3["ratio"]) * 2

In [ ]:
Convert(userId,front_image, side_image, top_image,fratio, sratio, tratio)

In [ ]:
print("please wait generating 3D view...") 
start = time.time() 
scad_filename = "static/" + userId + '/' + userId + ".1.scad"
stl_filename = "static/" + userId + '/' + userId + ".stl"
pcd_filename = "static/" + userId + '/' + userId + ".pcd"
if(os.path.isfile(scad_filename)):
    run("openscad.exe -o " + stl_filename + " " + scad_filename)
    if(os.path.isfile(stl_filename)):
        print("stl file generated at " + stl_filename)
        mesh = o3d.io.read_triangle_mesh(stl_filename)
        pointcloud = mesh.sample_points_poisson_disk(100000)
        o3d.io.write_point_cloud(pcd_filename, pointcloud)
        model = PyntCloud.from_file(pcd_filename)
        print("pcd file generated at " + pcd_filename)
        model.plot() 
        end = time.time() 
        print("Total time taken to convert from stl to point cloud:",end-start)
    else:
        print("file not found")
else:
    print("scad file not found")